In [1]:
# Function (tools) calling

In [2]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [3]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

### Step 1 : Ask LLM to return an instantiated function  

In [14]:
# llm = get_llm("gpt_35_edenai")
llm = get_llm("gpt_4_azure")

messages = []


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add])
prompt = def_prompt(
    "Use the provided functions  to compute what is 11 + 12; execute the function "
)
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

debug(ai_msg)

2024-07-04 14:42:04.353 | INFO     | python.ai_core.llm:get_llm:405 - get LLM : gpt_4_azure - configurable: True


2024-07-04 14:42:05.020 | INFO     | python.ai_core.llm:get_configurable:361 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


/tmp/ipykernel_136462/943656908.py:24 <module>
    ai_msg: AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_TluYnXjedLkuncB4cK6oPh4t',
                    'function': {
                        'arguments': '{"a":11,"b":12}',
                        'name': 'add',
                    },
                    'type': 'function',
                },
            ],
        },
        response_metadata={
            'token_usage': {
                'completion_tokens': 17,
                'prompt_tokens': 81,
                'total_tokens': 98,
            },
            'model_name': 'gpt-4',
            'system_fingerprint': 'fp_811936bd4f',
            'finish_reason': 'tool_calls',
            'logprobs': None,
            'content_filter_results': {},
        },
        id='run-6e12adb5-039b-45c2-8ed9-ae10e5e60add-0',
        tool_calls=[
            {
                'name': 'add',
                'ar

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TluYnXjedLkuncB4cK6oPh4t', 'function': {'arguments': '{"a":11,"b":12}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 81, 'total_tokens': 98}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-6e12adb5-039b-45c2-8ed9-ae10e5e60add-0', tool_calls=[{'name': 'add', 'args': {'a': 11, 'b': 12}, 'id': 'call_TluYnXjedLkuncB4cK6oPh4t'}], usage_metadata={'input_tokens': 81, 'output_tokens': 17, 'total_tokens': 98})

In [15]:
debug(llm_with_tools)

/tmp/ipykernel_136462/3318825891.py:1 <module>
    llm_with_tools: RunnableBinding(
        bound=AzureChatOpenAI(
            name='gpt4-turbo',
            client=<openai.resources.chat.completions.Completions object at 0x7fc05023e6b0>,
            async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc0502544c0>,
            model_name='gpt4-turbo',
            temperature=0.0,
            openai_api_key=SecretStr('**********'),
            openai_proxy='',
            azure_endpoint='https://mutualizedopenai.openai.azure.com',
            deployment_name='gpt4-turbo',
            openai_api_version='2023-05-15',
            openai_api_type='azure',
        ),
        kwargs={
            'tools': [
                {
                    'type': 'function',
                    'function': {
                        'name': 'add',
                        'description': (
                            'Adds a and b.\n'
                            '\n'
            

RunnableBinding(bound=AzureChatOpenAI(name='gpt4-turbo', client=<openai.resources.chat.completions.Completions object at 0x7fc05023e6b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc0502544c0>, model_name='gpt4-turbo', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://mutualizedopenai.openai.azure.com', deployment_name='gpt4-turbo', openai_api_version='2023-05-15', openai_api_type='azure'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}}]})

In [5]:
# Step 2 : Execute the function, and send result to LLM

In [16]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

'add' tool called
/tmp/ipykernel_136462/3057862763.py:11 <module>
    r: AIMessage(
        content='The sum of 11 and 12 is 23.',
        response_metadata={
            'token_usage': {
                'completion_tokens': 13,
                'prompt_tokens': 83,
                'total_tokens': 96,
            },
            'model_name': 'gpt-4',
            'system_fingerprint': 'fp_811936bd4f',
            'finish_reason': 'stop',
            'logprobs': None,
            'content_filter_results': {},
        },
        id='run-5cc5466d-e309-48ff-83b5-e85aca927e90-0',
        usage_metadata={
            'input_tokens': 83,
            'output_tokens': 13,
            'total_tokens': 96,
        },
    ) (AIMessage)


AIMessage(content='The sum of 11 and 12 is 23.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 83, 'total_tokens': 96}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-5cc5466d-e309-48ff-83b5-e85aca927e90-0', usage_metadata={'input_tokens': 83, 'output_tokens': 13, 'total_tokens': 96})

###  Other method : Use a tool calling agent

In [22]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)"""
    print("exponentiate")
    return math.pow(x, y)


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
debug(r)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 12, 'b': 1000000}`


1000012
Invoking: `exponentiate` with `{'x': 100, 'y': 3}`


exponentiate
1000000.0The result of \( 12 + 100^3 \) is 1,000,012.

> Finished chain.
/tmp/ipykernel_136462/800412656.py:39 <module>
    r: {
        'input': 'what is 12  + 100^3',
        'output': 'The result of \\( 12 + 100^3 \\) is 1,000,012.',
    } (dict) len=2


{'input': 'what is 12  + 100^3',
 'output': 'The result of \\( 12 + 100^3 \\) is 1,000,012.'}

In [8]:
## Call Eden.ai provided tools